In [ ]:
import pprint
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pyspiel
from open_spiel.python.algorithms import deep_cfr
print(pyspiel.__version__())

2026-01-21 12:05:14.694808: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2026-01-21 12:05:27.348249: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Instructions for updating:
non-resource variables are not supported in the long term


ImportError: cannot import name 'deep_cfr' from 'open_spiel.python.algorithms' (/workspaces/codespaces-blank/ovenv/lib/python3.12/site-packages/open_spiel/python/algorithms/__init__.py)

In [10]:
save_path = ""

In [2]:
# Game configuration
game_string = "liars_dice"
game_config = {"players": 2, "numdice": 1}
game = pyspiel.load_game(game_string, game_config)

print("Loaded game:", game_string, "with config:", game_config)
print("Game info: players =", game.num_players(), ", distinct actions =", game.num_distinct_actions())

Loaded game: liars_dice with config: {'players': 2, 'numdice': 1}
Game info: players = 2 , distinct actions = 13


In [4]:
# Solver initiation
sess = tf.Session()
# solver_kwargs = dict(
#     session=sess,
#     game=game,
#     policy_network_layers=(100, 100),        # smaller for quicker runs; increase for quality
#     advantage_network_layers=(100, 100),
#     num_iterations=30,           # outer iterations
#     num_traversals=10,           # traversals per iteration per player
#     learning_rate= 0.002,
#     batch_size_advantage=10,
#     batch_size_strategy=10,
#     memory_capacity=50000,
#     policy_network_train_steps=1,
#     advantage_network_train_steps=1,
#     reinitialize_advantage_networks=True
# )

solver_kwargs = dict(
    session=sess,
    game=game,
    # (64, 64) is usually sufficient for 1-die Liar's Dice
    policy_network_layers=(64, 64),
    advantage_network_layers=(64, 64),
    # Total outer loops. 
    # Deep CFR converges in fewer iterations than tabular CFR, but each is slower.
    num_iterations=200,
    # How many games to simulate per iteration to fill the buffer.
    # Needs to be higher to capture the variance of dice rolls.
    num_traversals=200,
    # Standard Adam learning rate
    learning_rate=1e-3,
    # Batch sizes must be larger to get a stable gradient estimate
    batch_size_advantage=128,
    batch_size_strategy=1024,
    memory_capacity=100000,
    # CRITICAL CHANGE: 
    # Since reinitialize=True, the network is wiped clean every iteration.
    # It needs hundreds of steps to learn the new data in the buffer from scratch.
    advantage_network_train_steps=400,
    # CRITICAL CHANGE:
    # In this implementation, the policy network is trained ONLY ONCE 
    # at the very end of solve(). It needs thousands of steps to fit the average strategy.
    policy_network_train_steps=5000,
    reinitialize_advantage_networks=True
)


solver = deep_cfr.DeepCFRSolver(**solver_kwargs)

2025-12-25 02:27:36.162134: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [50]:
sess.run(tf.global_variables_initializer())

In [51]:
# Train
print("\nStarting solve() ... (this may take a while depending on params)")
policy_network, adv_losses, policy_loss = solver.solve()
print("\nDone training.")


Starting solve() ... (this may take a while depending on params)


2025-11-29 02:30:52.026670: W tensorflow/c/c_api.cc:305] Operation '{name:'mlp_22/weights_2/Assign' id:7163 op device:{requested: '', assigned: ''} def:{{{node mlp_22/weights_2/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@mlp_22/weights_2"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](mlp_22/weights_2, mlp_22/truncated_normal_2)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2025-11-29 02:30:56.278075: W tensorflow/c/c_api.cc:305] Operation '{name:'mlp_23/weights_2/Assign' id:7202 op device:{requested: '', assigned: ''} def:{{{node mlp_23/weights_2/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@mlp_23/weights_2"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](mlp_23/weights_2, mlp_23/truncated_normal_2)}}' was changed by setting attribute after it was run by a s


Done training.


In [84]:
# test:
for i in range(6):
    state = game.new_initial_state()
    while state.is_chance_node():
        state.apply_action(i)
    print(state.information_state_string())
    for a in range(6):
        print(round(solver.action_probabilities(state)[a], 2), end = ' ')
    print()
# for i in range(8):
#     state.apply_action(i+1)
# action_probs = solver.action_proba\bilities(state)
# print("\nAction probabilities at initial state (only legal actions printed):")
# for a, p in action_probs.items():
#     print(f"  Action {a}: prob = {p:.4f}")

1
0.32 0.16 0.25 0.14 0.07 0.0 
2
0.09 0.39 0.19 0.17 0.13 0.0 
3
0.08 0.1 0.44 0.16 0.17 0.0 
4
0.1 0.1 0.06 0.64 0.07 0.0 
5
0.04 0.14 0.06 0.02 0.72 0.01 
6
0.13 0.29 0.2 0.22 0.15 0.01 


In [68]:
# dir(solver)
tbl = solver.to_tabular()

In [73]:
# try:
#     print("Policy network:", policy_network)
# except Exception:
#     pass

# Basic diagnostics
print("\nDiagnostics:")
print("  Environment steps (tree traversals) =", solver.get_environment_steps())
print("  Advantage losses collected per player (lengths):",
      {p: len(adv_losses[p]) for p in adv_losses})
print("  Final policy network loss (if computed):", policy_loss)


Diagnostics:
  Environment steps (tree traversals) = 7040810
  Advantage losses collected per player (lengths): {0: 200, 1: 200}
  Final policy network loss (if computed): 1.1314746


In [13]:
# Save
now = datetime.now()
save_path = f"SAVED-{now.strftime("%Y-%m-%d-%H-%M-%S")}"
saver = tf.train.Saver()
saver.save(sess, save_path)
print("\nSaved model checkpoint to:", save_path)

SyntaxError: f-string: unmatched ')' (721806739.py, line 3)

In [12]:
from datetime import datetime

# Get current date and time
now = datetime.now()
print(now.strftime("%Y-%m-%d-%H-%M-%S"))

2025-12-06-21-55-45


In [ ]:
sess.close()

In [56]:
dir(state)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'action_to_string',
 'apply_action',
 'apply_action_with_legality_check',
 'apply_actions',
 'apply_actions_with_legality_checks',
 'chance_outcomes',
 'child',
 'clone',
 'current_player',
 'distribution_support',
 'full_history',
 'get_game',
 'get_type',
 'history',
 'history_str',
 'information_state_string',
 'information_state_tensor',
 'is_chance_node',
 'is_initial_state',
 'is_mean_field_node',
 'is_player_node',
 'is_simultaneous_node',
 'is_terminal',
 'legal_actions',
 'legal_actions_mask',
 'mean_field_population',
 'move_number',
 'num_distinct_actions',
 'num_players',
 'observation_string',
 'observat

In [75]:
state = game.new_initial_state()
state.chance_outcomes()

[(0, 0.16666666666666666),
 (1, 0.16666666666666666),
 (2, 0.16666666666666666),
 (3, 0.16666666666666666),
 (4, 0.16666666666666666),
 (5, 0.16666666666666666)]

In [3]:
class Act:
    def __init__(self, bid):
        self.bid = bid
    def to_action(self):
        bid = self.bid
        if len(bid) == 3 and bid[0].isdigit() and bid[1] == '-' and bid[2].isdigit():
            return (int(bid[0])-1) * 6 + int(bid[2]) - 1
        else:
            return 12
def read_act():
    return Act(input())

def act2str(action):
    if action == 12:
        return "Liar!"
    else:
        return f"{(action) // 6 + 1}-{action%6 + 1}"

def solver_act(state):
    d = solver.action_probabilities(state)
    values = []
    weights = []
    for x in d:
        values.append(x)
        weights.append(d[x])
    return random.choices(values, weights=weights, k=1)[0]

import random
ans = None
wins = 0
losses = 0
while True:
    print("----------------------------------------------")
    print("New game")
    player_id = random.choice([0, 1])
    if player_id == 0:
        print("You go first")
    else:
        print("You go second")
    state = game.new_initial_state()
    for i in range(2):
        values, weights = zip(*state.chance_outcomes())
        action = random.choices(values, weights=weights, k=1)[0]
        state.apply_action(action)
    print(f"Your dice is {state.history()[player_id] + 1}")
    while not state.is_terminal():
        if state.current_player() == player_id:
            # if (len(state.history()) < 4):
            #     print(f"Your dice is {state.information_state_string()[0]}")
            # print("Current info state:")
            # print(state.information_state_string()[0])
            print("Your bid is:", end = ' ')
            act = read_act()
            state.apply_action(act.to_action())
        else:
            act = solver_act(state)
            print(act2str(act))
            state.apply_action(act)
    print(f"His cube was: {state.history()[1-player_id] + 1}")
    # print(state.rewards())
    if state.rewards()[player_id] == 1:
        print("You won!")
        wins += 1
    else:
        print("You lost!")
        losses += 1
    print(f"History: {state.history_str()}")

----------------------------------------------
New game
You go first
Your dice is 6
Your bid is: 

 1-2


NameError: name 'solver' is not defined

In [107]:
state.history()

[1, 2]

In [142]:
print(losses, wins)

2 2


In [116]:
dir(state)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'action_to_string',
 'apply_action',
 'apply_action_with_legality_check',
 'apply_actions',
 'apply_actions_with_legality_checks',
 'chance_outcomes',
 'child',
 'clone',
 'current_player',
 'distribution_support',
 'full_history',
 'get_game',
 'get_type',
 'history',
 'history_str',
 'information_state_string',
 'information_state_tensor',
 'is_chance_node',
 'is_initial_state',
 'is_mean_field_node',
 'is_player_node',
 'is_simultaneous_node',
 'is_terminal',
 'legal_actions',
 'legal_actions_mask',
 'mean_field_population',
 'move_number',
 'num_distinct_actions',
 'num_players',
 'observation_string',
 'observat

In [143]:
def to_action():
    bid = input()
    if len(bid) == 3 and bid[0].isdigit() and bid[1] == '-' and bid[2].isdigit():
        return (int(bid[0])-1) * 6 + int(bid[2]) - 1
    else:
        return 12
def act2str(action):
    if action == 12:
        return "Liar!"
    else:
        return f"{(action) // 6 + 1}-{action%6 + 1}"
while True:
    wer = to_action()
    print(wer, act2str(wer))

In [152]:
for i in range(1):
    state = game.new_initial_state()
    state.apply_action(i) ## solver gets 3
    state.apply_action(2) ## opponent gets smth
    state.apply_action(3) ## opponent bets 1-4
    print(solver.action_probabilities(state))

{4: 0.53899896, 5: 0.020347951, 6: 0.0023065552, 7: 0.0043633627, 8: 0.010817973, 9: 0.008835696, 10: 0.0005243021, 11: 0.00021154173, 12: 0.40143138}


In [158]:
h = [5, 0, 0, 3, 5, 6, 12]
state = game.new_initial_state()
for x in h:
    print(state.action_to_string(x))
    state.apply_action(x)
# print(state.information_state())

Roll 6
Roll 1
1-1
1-4
1-6
2-1
Liar


In [159]:
state.rewards()

[-1.0, 1.0]

In [160]:
dir(game)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'action_to_string',
 'deserialize_state',
 'get_parameters',
 'get_type',
 'information_state_tensor_layout',
 'information_state_tensor_shape',
 'information_state_tensor_size',
 'make_observer',
 'max_chance_nodes_in_history',
 'max_chance_outcomes',
 'max_game_length',
 'max_history_length',
 'max_move_number',
 'max_utility',
 'min_utility',
 'new_initial_state',
 'new_initial_state_for_population',
 'new_initial_states',
 'num_distinct_actions',
 'num_players',
 'observation_tensor_layout',
 'observation_tensor_shape',
 'observation_tensor_size',
 'policy_tensor_shape',
 'utility_sum']